In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as img
import numpy as np
import os
import collections
from collections import defaultdict
from shutil import copy
from shutil import copytree, rmtree
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
# from autoaugment import ImageNetPolicy

In [2]:
from tqdm import tqdm

In [3]:
# Check the extracted dataset folder
os.listdir('food-101/food-101/')

['README.txt', 'train', 'meta', 'test', 'images']

In [4]:
src_train = 'food-101/food-101/train'
src_test = 'food-101/food-101/test'
# torch.set_default_tensor_type(torch.cuda.FloatTensor)
torch.set_default_tensor_type(torch.FloatTensor)

In [5]:

# TODO: Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(256),
                                       transforms.RandomHorizontalFlip(),#ImageNetPolicy(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(src_train, transform=train_transforms)
test_data = datasets.ImageFolder(src_test, transform=test_transforms)
testdata=datasets.ImageFolder(src_test, transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size= 128)

In [6]:
len(trainloader)

1184

In [7]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model = models.vgg19(pretrained=True).to(device)
model


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [8]:
pytorch_total_params = sum(p.numel() for p in model.parameters())
pytorch_total_params

143667240

In [9]:
for param in model.parameters():
    param.requires_grad = True

In [10]:
model.classifier[6] = nn.Sequential(
                      nn.Linear(4096, 256), 
                      nn.ReLU(), 
                      nn.Dropout(0.4),
                      nn.Linear(256, 101),                   
                      nn.LogSoftmax(dim=1)).to(device)

In [11]:
model.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Sequential(
    (0): Linear(in_features=4096, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.4, inplace=False)
    (3): Linear(in_features=256, out_features=101, bias=True)
    (4): LogSoftmax()
  )
)

In [12]:
import time

In [13]:


criterion = nn.CrossEntropyLoss()

In [14]:
optimizer = optim.Adam(model.parameters(), lr=0.001, betas=[0.9, 0.999])
# optimizer = optim.Adam(model.classifier.parameters(), lr=0.001, betas=[0.9, 0.999])

In [15]:
import numpy as np
import time
def train(n_epochs,trainloader,testloader, net, optimizer, criterion, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    running_loss=0
    train_loss = []
    val_loss = []
    accu = []
    val_accu = []
  
    for epoch in range(n_epochs):
        start_epoch = time.time()
        
        train_accuracy = 0
        for inputs, labels in tqdm(trainloader):
            
        # Move input and label tensors to the default device
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            start = time.time()
            logps = net(inputs)
            loss = criterion(logps, labels)
            loss.backward()
            optimizer.step()
            top_p, top_class = logps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            train_accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
            
        
            running_loss += loss.item()
        
        train_loss.append(running_loss/len(trainloader))
        accu.append(train_accuracy/len(trainloader))
        
        net.eval()
        valid_loss=0
        valid_accuracy=0
        with torch.no_grad():
            for inputs, labels in tqdm(testloader):
                inputs, labels = inputs.to(device), labels.to(device)
                logps = net(inputs)
                batch_loss = criterion(logps, labels)
                valid_loss += batch_loss.item()
                    
                    # Calculate accuracy
                
                top_p, top_class = logps.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                valid_accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
           
        
            if valid_loss <= valid_loss_min:
                print("Validation loss decreased  Saving model")
                torch.save(net.state_dict(),'food_classifier_VGG19.pt')
                valid_loss_min=valid_loss
                
                    
                
                  
                
                
                   
            val_loss.append(valid_loss/len(testloader))
            val_accu.append(valid_accuracy/len(testloader))
            
            print(f"Device = cuda; Time per batch: {(time.time() - start):.3f} seconds")       
            print(f"Device = cuda; Time per epoch: {(time.time() - start_epoch)/60:.3f} minutes")       
            print(f"Epoch {epoch}/{n_epochs}.. "
                  f"Train loss: {running_loss/len(trainloader):.3f}.. "
                  f"Test loss: {valid_loss/len(testloader):.3f}.. "
                  f"Train accuracy: {train_accuracy/len(trainloader):.3f}.."
                  f"Test accuracy: {valid_accuracy/len(testloader):.3f}")
            running_loss = 0
            net.train()
    return train_loss, val_loss, accu, val_accu

In [ ]:
train_loss, val_loss, accu, val_accu = train(10, trainloader, testloader, model, optimizer, criterion,'model_vowel_consonant.pt')

100%|██████████| 198/198 [04:31<00:00,  1.37s/it]


Validation loss decreased  Saving model


  0%|          | 0/1184 [00:00<?, ?it/s]

Device = cuda; Time per batch: 273.105 seconds
Device = cuda; Time per epoch: 32.598 minutes
Epoch 0/10.. Train loss: 4.618.. Test loss: 4.616.. Train accuracy: 0.009..Test accuracy: 0.010


  7%|▋         | 78/1184 [01:50<26:26,  1.43s/it]

In [ ]:
def plot_accuracy(val_acc, train_acc, title):
    plt.title(title)
    plt.plot(train_acc) 
    plt.plot(val_acc)
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train_accuracy', 'validation_accuracy'], loc='best')
    plt.show()


def plot_loss(val_loss, train_loss, title):
    plt.title(title)
    plt.plot(train_loss)
    plt.plot(val_loss)
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train_loss', 'validation_loss'], loc='best')
    plt.show()


plot_accuracy(val_accu, accu, 'FOOD101-ResNet18')
plot_loss(val_loss, train_loss, 'FOOD101-ResNet18')